| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | [lambdas](./07_lambdas.ipynb) |
| ----------------------------------- | -------------------------------------------- | ------------- | --------------------------------------- | --------------------------------- | --------------------------------- | --------------------------------------- | ------------------------------- | ----------------------------- |

# Acccessing and Indexing Geochemical Data with pyrolite

The `pyrolite.pyrochem` API provides access to indexing and transformation functions. This allows easy subsetting of geochemical datasets which can otherwise be unweildly (expecially as the number of columns increases..). To provide a simple illustration we generate a synthetic dataset to work from, which contains an array of typical geochemical measures - oxide components, element components (here as ppm), element ratios and isotope ratios. While this size dataset is managable, some of the indexing tools pyrolite provides make it straightforward to pull out different parts of the dataset.

A key thing to note here is that `pyrolite` provides a range of built-in functionality around known elements, oxides and isotope ratios. Typically, this means that having column names which are names of elements, oxide components or isotope ratios in the form $^{xx}A/^{xy}B$. This means you'll need to keep track of units etc yourself - which is where comments can be useful. This aspect may change in the future, with some ability to deal with units, non-standard column names and potentially even molecular ions (e.g. for hydrogeochemistry).

First let's create some data to play with:

In [1]:
import pyrolite.geochem
import pandas as pd
import numpy as np
from pyrolite.util.synthetic import normal_frame

pd.set_option("precision", 3)  # smaller graphical outputs
 
df = normal_frame(columns=['CaO', 'MgO', 'SiO2', 'FeO','Na2O', 'Ni', 'Ti', 'La', 'Lu', 'Te', 'Y']) * 100 # create a dataframe which contains some oxides and elements, which sum to 100%
df[['Ni', 'Ti', 'La', 'Lu', 'Te', 'Y']] *= 10 # simulate some ppm data in a fairly blunt way
df['Sr87/Sr86'] = 0.0700  / 0.0986 + np.random.randn(df.index.size) * 0.0001 # simulate some Sr isotope data 
df

,CaO,MgO,SiO2,FeO,Na2O,Ni,Ti,La,Lu,Te,Y,Sr87/Sr86
0,8.451,25.527,15.105,4.968,10.545,64.603,25.975,117.380,23.073,14.078,108.926,0.71
1,8.670,25.345,15.091,5.233,10.658,60.026,23.958,114.765,23.064,16.277,111.936,0.71
2,8.309,26.174,15.181,4.653,10.246,63.165,27.252,117.867,23.043,15.280,107.755,0.71
3,8.197,26.415,14.968,4.953,10.430,65.751,27.531,119.907,22.845,13.905,100.435,0.71
4,8.634,26.129,15.299,4.435,10.271,65.219,27.316,120.711,21.867,13.932,103.278,0.71
5,7.573,27.604,14.990,4.952,10.695,63.611,22.269,110.065,23.533,13.433,108.961,0.71
6,8.581,26.943,15.252,4.192,10.185,63.911,25.051,122.504,23.336,13.656,100.011,0.71
7,8.792,25.960,15.407,4.826,10.439,60.616,25.130,114.376,22.287,16.674,106.676,0.71
8,8.481,26.239,15.233,4.975,10.546,62.020,26.442,116.792,22.260,15.104,102.632,0.71
9,8.259,24.690,14.811,5.343,10.543,67.919,28.058,122.302,23.699,12.823,108.737,0.71


To save a bit of space, below we'll use `df.head(2)` to restrict the output to just the top two rows:

In [2]:
df.head(2)

,CaO,MgO,SiO2,FeO,Na2O,Ni,Ti,La,Lu,Te,Y,Sr87/Sr86
0,8.451,25.527,15.105,4.968,10.545,64.603,25.975,117.380,23.073,14.078,108.926,0.71
1,8.670,25.345,15.091,5.233,10.658,60.026,23.958,114.765,23.064,16.277,111.936,0.71


### Selecting Data

To select a subset of columns from the dataframe, the `pyrolite.pyrochem` API has a few built-in selectors:

In [3]:
df.head(2).pyrochem.oxides

,CaO,MgO,SiO2,FeO,Na2O
0,8.451,25.527,15.105,4.968,10.545
1,8.670,25.345,15.091,5.233,10.658


In [4]:
df.head(2).pyrochem.elements

,Ni,Ti,La,Lu,Te,Y
0,64.603,25.975,117.380,23.073,14.078,108.926
1,60.026,23.958,114.765,23.064,16.277,111.936


In [5]:
df.head(2).pyrochem.REE

,La,Lu
0,117.380,23.073
1,114.765,23.064


In [6]:
df.head(2).pyrochem.REY

,La,Y,Lu
0,117.380,108.926,23.073
1,114.765,111.936,23.064


In [7]:
df.head(2).pyrochem.compositional

,CaO,MgO,SiO2,FeO,Na2O,Ni,Ti,La,Lu,Te,Y
0,8.451,25.527,15.105,4.968,10.545,64.603,25.975,117.380,23.073,14.078,108.926
1,8.670,25.345,15.091,5.233,10.658,60.026,23.958,114.765,23.064,16.277,111.936


In [8]:
df.head(2).pyrochem.isotope_ratios

,Sr87/Sr86
0,0.71
1,0.71


---
These selectors can also be used to re-assign modified subsets back to the dataframe - useful if you wanted to just process/scale/transform/standardise a subset of the data:

In [9]:
df.pyrochem.isotope_ratios /= 0.7016

In [10]:
df.head(2).pyrochem.isotope_ratios

,Sr87/Sr86
0,1.012
1,1.012


------
### Listing Columns
If you just want the column names, it's easy enough to get those too (these are actually used in the indexing above):

In [11]:
df.pyrochem.list_oxides

['CaO', 'MgO', 'SiO2', 'FeO', 'Na2O']

In [12]:
df.pyrochem.list_elements

['Ni', 'Ti', 'La', 'Lu', 'Te', 'Y']

In [13]:
df.pyrochem.list_REE

['La', 'Lu']

In [14]:
df.pyrochem.list_compositional

['CaO', 'MgO', 'SiO2', 'FeO', 'Na2O', 'Ni', 'Ti', 'La', 'Lu', 'Te', 'Y']

In [15]:
df.pyrochem.list_isotope_ratios

['Sr87/Sr86']

----
<div class='alert alert-warning'> <font size="+1" color="black"><b> Checkpoint & Time Check</b><br>How are things going?</font></div>

----

| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | [lambdas](./07_lambdas.ipynb) |
| ----------------------------------- | -------------------------------------------- | ------------- | --------------------------------------- | --------------------------------- | --------------------------------- | --------------------------------------- | ------------------------------- | ----------------------------- |